In [1]:
"""
teachopencadd
"""
import time
import random
from pathlib import Path

import pandas as pd
import numpy
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdFingerprintGenerator

from teachopencadd.utils import seed_everything

from pathlib import Path

from rdkit.Chem import (
    PandasTools,
    Draw,
    Descriptors,
    MACCSkeys,
    rdFingerprintGenerator,
)

%load_ext autoreload
%autoreload 2
from utils import  tanimoto_distance_matrix_from_centroid
from utils import  tanimoto_distance_matrix, cluster_fingerprints

# seed_everything()  # fix seed to get deterministic outputs

In [2]:
data_df = pd.read_csv("../raw/herg_raw_0528_clean.csv")

In [3]:
data_df.columns

Index(['Molecule ChEMBL ID', 'y', 'smiles'], dtype='object')

In [4]:
data_df["MW"]=data_df["smiles"].apply(lambda x : Descriptors.ExactMolWt(Chem.MolFromSmiles(x)))

In [5]:
PandasTools.AddMoleculeColumnToFrame(data_df, smilesCol="smiles")
PandasTools.RenderImagesInAllDataFrames(True)

In [6]:
data_df["maccs"] = data_df.ROMol.apply(MACCSkeys.GenMACCSKeys)
fps_1 = data_df["maccs"].tolist()
clusters = cluster_fingerprints(fps_1, cutoff=0.4)
print(len(clusters))


605


In [ ]:
'''
Test 1 : clusters[i]) >2
Test 2 : clusters[i]) >5
'''
cls_less_list = []
cls_more_list = []
# len(clusters[0])
for i in range(len(clusters)):
    if len(clusters[i]) >5:
        cls_more_list.append(i)
    elif len(clusters[i]) <6:
        cls_less_list.append(i)

In [8]:
set(cls_less_list) & set(cls_more_list)

set()

In [9]:
len(cls_less_list)

422

In [10]:
len(cls_more_list)

183

In [11]:

cls_more_dfs = []
for i in (cls_more_list):
    # print(len(clusters[i]))
    df = data_df.loc[list(clusters[i])]
    df["Cluster"]=i
    df.reset_index(drop=True,inplace=True)
    cls_more_dfs.append(df)

In [12]:
cls_more_df = pd.concat(cls_more_dfs)
cls_more_df.shape

(14961, 7)

In [13]:
cls_more_df["Cluster"].value_counts()

Cluster
0      7271
1       904
2       832
3       488
4       320
       ... 
165       6
164       6
163       6
162       6
182       6
Name: count, Length: 183, dtype: int64

In [14]:
cls_more_df.y.value_counts()

y
0    9140
1    5821
Name: count, dtype: int64

In [16]:
cls_more_df[(cls_more_df['MW'] >= 300) & (cls_more_df['MW'] <= 500)].y.value_counts()

y
0    6281
1    4262
Name: count, dtype: int64

In [17]:

cls_less_dfs = []
for i in cls_less_list:
    # print(len(clusters[i]))
    df = data_df.loc[list(clusters[i])]
    df["Cluster"]=i
    df.reset_index(drop=True,inplace=True)
    cls_less_dfs.append(df)

In [18]:
cls_less_df = pd.concat(cls_less_dfs)
cls_less_df.shape

(845, 7)

In [19]:
cls_less_df[(cls_less_df['MW'] >= 300) & (cls_less_df['MW'] <= 500)].y.value_counts()

y
0    275
1    104
Name: count, dtype: int64

In [20]:
cls_more_df.columns

Index(['Molecule ChEMBL ID', 'y', 'smiles', 'MW', 'ROMol', 'maccs', 'Cluster'], dtype='object')

In [ ]:
all_data_cls = pd.concat([cls_more_df,cls_less_df])[["smiles","y","MW","Cluster"]]

In [ ]:
all_data_cls.to_csv("All_Data_Cluster.csv",index=None)

In [24]:
cls_less_df["split"]="test"

In [ ]:
cls_less_df[["smiles","y","split"]].to_csv("Test_2.csv",index=None)
#cls_less_df[["smiles","y","split"]].to_csv("Test_1.csv",index=None)

In [ ]:
cls_more_df[["smiles","y","Cluster"]].to_csv("Train_Val_Cluster.csv",index=None)